In [ ]:
import requests
import time
import csv

API_KEY = '9e9ab06256174d82a01dc73f02418ca2'  # Reemplaza esto por tu API KEY de RAWG
BASE_URL = 'https://api.rawg.io/api/games'

def obtener_videojuegos_con_rating(max_paginas=100):
    juegos = []
    params = {
        'key': API_KEY,
        'dates': '1995-01-01,2024-12-31',
        'ordering': '-released',
        'page_size': 40
    }

    for pagina in range(1, max_paginas + 1):
        params['page'] = pagina
        response = requests.get(BASE_URL, params=params)
        if response.status_code != 200:
            break

        data = response.json()
        for juego in data.get('results', []):
            rating = juego.get('rating')
            if rating is not None and rating > 0:  # aseguramos que tenga puntuación válida
                juegos.append({
                    'nombre': juego.get('name'),
                    'fecha_lanzamiento': juego.get('released'),
                    'puntuacion_media': rating,
                    'generos': ', '.join([g['slug'] for g in juego.get('genres', [])])
                })

        if not data.get('next'):
            break

        time.sleep(1)  # buena práctica para evitar bloqueos por rate limit

    return juegos

def guardar_csv(datos, nombre_archivo='videojuegos_con_puntuacion.csv'):
    with open(nombre_archivo, 'w', newline='', encoding='utf-8') as f:
        columnas = ['nombre', 'fecha_lanzamiento', 'puntuacion_media', 'generos']
        writer = csv.DictWriter(f, fieldnames=columnas)
        writer.writeheader()
        writer.writerows(datos)

# Ejecutar
if __name__ == '__main__':
    datos = obtener_videojuegos_con_rating(max_paginas=100)
    guardar_csv(datos)


In [ ]:
import pandas as pd
import plotly.express as px

# Cargar los datos
df = pd.read_csv('videojuegos_con_puntuacion.csv')

# Asegurarse de que hay datos válidos
df = df.dropna(subset=['fecha_lanzamiento', 'puntuacion_media', 'generos'])

# Extraer el año
df['año'] = pd.to_datetime(df['fecha_lanzamiento'], errors='coerce').dt.year
df = df.dropna(subset=['año'])

# Para simplificar, usamos solo el primer género si hay varios
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'desconocido')

# Crear gráfico interactivo
fig = px.scatter(
    df,
    x='año',
    y='puntuacion_media',
    color='genero_principal',
    hover_name='nombre',
    title='🎮 Puntuaciones medias de videojuegos por año y género',
    labels={
        'año': 'Año de lanzamiento',
        'puntuacion_media': 'Puntuación media',
        'genero_principal': 'Género principal'
    },
    size_max=15,
    opacity=0.7
)

fig.update_layout(template='plotly_dark')
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Cargar los datos desde el CSV
df = pd.read_csv('videojuegos_con_puntuacion.csv')

# Eliminar registros sin fecha o puntuación
df = df.dropna(subset=['fecha_lanzamiento', 'puntuacion_media', 'generos'])

# Convertir fecha y extraer año
df['año'] = pd.to_datetime(df['fecha_lanzamiento'], errors='coerce').dt.year
df = df.dropna(subset=['año'])

# Filtrar por el rango de años
df = df[(df['año'] >= 1995) & (df['año'] <= 2024)]

# Obtener solo el primer género (por simplicidad)
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'desconocido')

# Crear gráfico interactivo
fig = px.scatter(
    df,
    x='año',
    y='puntuacion_media',
    color='genero_principal',
    hover_name='nombre',
    title='🎮 Puntuaciones medias de videojuegos (1995–2024) por género',
    labels={
        'año': 'Año de lanzamiento',
        'puntuacion_media': 'Puntuación media',
        'genero_principal': 'Género principal'
    },
    opacity=0.7
)

fig.update_layout(template='plotly_dark')
fig.show()


In [ ]:
import requests
import time
import csv

API_KEY = '9e9ab06256174d82a01dc73f02418ca2'  # ← REEMPLÁZALO
BASE_URL = 'https://api.rawg.io/api/games'

def obtener_juegos_desde_1995(paginas_por_año=3):
    juegos_por_año = []

    for año in range(1995, 2025):  # de 1995 a 2024 inclusive
        juegos_este_año = []
        for pagina in range(1, paginas_por_año + 1):
            params = {
                'key': API_KEY,
                'dates': f'{año}-01-01,{año}-12-31',
                'page': pagina,
                'page_size': 40,
                'ordering': '-rating'
            }
            response = requests.get(BASE_URL, params=params)
            if response.status_code != 200:
                print(f"Error {response.status_code} en el año {año}, página {pagina}")
                break

            data = response.json()
            for juego in data.get('results', []):
                rating = juego.get('rating')
                if rating and rating > 0 and juego.get('genres'):
                    juegos_este_año.append({
                        'nombre': juego.get('name'),
                        'fecha_lanzamiento': juego.get('released'),
                        'año': año,
                        'puntuacion_media': rating,
                        'generos': ', '.join([g['slug'] for g in juego['genres']])
                    })

            if not data.get('next'):
                break
            time.sleep(1)  # evitar bloqueo

        if not juegos_este_año:
            # Añadir entrada vacía si no hay datos para el año
            juegos_por_año.append({
                'nombre': 'Sin datos',
                'fecha_lanzamiento': None,
                'año': año,
                'puntuacion_media': None,
                'generos': 'sin_datos'
            })
        else:
            juegos_por_año.extend(juegos_este_año)

    return juegos_por_año

def guardar_csv(datos, archivo='videojuegos_1995_2024_completo.csv'):
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        campos = ['nombre', 'fecha_lanzamiento', 'año', 'puntuacion_media', 'generos']
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        writer.writerows(datos)

# Ejecutar
if __name__ == '__main__':
    juegos = obtener_juegos_desde_1995(paginas_por_año=3)
    guardar_csv(juegos)


In [ ]:
import pandas as pd
import plotly.express as px

# Cargar el archivo CSV
df = pd.read_csv('videojuegos_1995_2024_completo.csv')

# Filtrar filas que sí tienen puntuación válida
df_validos = df.dropna(subset=['puntuacion_media', 'año'])

# Asegurar tipo numérico
df_validos['año'] = df_validos['año'].astype(int)

# ----------------------------
# 📈 Gráfico: Número de juegos por año
# ----------------------------
conteo = df_validos.groupby('año').size().reset_index(name='cantidad_juegos')

fig1 = px.bar(
    conteo,
    x='año',
    y='cantidad_juegos',
    title='🎮 Número de videojuegos puntuados por año (1995–2024)',
    labels={'cantidad_juegos': 'Número de juegos'},
)
fig1.update_layout(template='plotly_dark')
fig1.show()

# ----------------------------
# 📊 Gráfico: Puntuación media por año
# ----------------------------
promedios = df_validos.groupby('año')['puntuacion_media'].mean().reset_index()

fig2 = px.line(
    promedios,
    x='año',
    y='puntuacion_media',
    markers=True,
    title='⭐ Puntuación media de videojuegos por año (1995–2024)',
    labels={'puntuacion_media': 'Puntuación media'}
)
fig2.update_layout(template='plotly_dark')
fig2.show()

In [ ]:
import pandas as pd
import plotly.express as px

# Cargar los datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')

# Filtrar juegos con datos válidos
df = df.dropna(subset=['puntuacion_media', 'año', 'generos'])
df['año'] = df['año'].astype(int)

# Tomar solo el primer género de la lista
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

# Agrupar por año y género, y calcular media de puntuación
pivot = df.groupby(['genero_principal', 'año'])['puntuacion_media'].mean().reset_index()

# Convertir a formato matriz para heatmap
heatmap_data = pivot.pivot(index='genero_principal', columns='año', values='puntuacion_media')

# Crear heatmap
fig = px.imshow(
    heatmap_data,
    labels=dict(x="Año", y="Género", color="Puntuación media"),
    title="🔥 Mapa de calor de la puntuación media por género y año (1995–2024)",
    text_auto=".2f",
    aspect="auto",
    color_continuous_scale="Viridis"
)

fig.update_layout(template="plotly_dark")
fig.show()


In [ ]:
import pandas as pd
from scipy.stats import linregress

# Cargar datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')
df = df.dropna(subset=['puntuacion_media', 'año', 'generos'])
df['año'] = df['año'].astype(int)
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

# Agrupar por año y género
grupo = df.groupby(['genero_principal', 'año'])['puntuacion_media'].mean().reset_index()

# Clasificar tendencia por género
clasificacion = []

for genero in grupo['genero_principal'].unique():
    datos = grupo[grupo['genero_principal'] == genero]

    if len(datos['año'].unique()) < 5:
        # No hay suficientes datos para detectar tendencia
        clasificacion.append((genero, 'insuficiente'))
        continue

    # Regresión lineal: puntuación media ~ año
    slope, _, _, p_value, _ = linregress(datos['año'], datos['puntuacion_media'])

    if abs(slope) < 0.01:
        tendencia = 'estable'
    elif slope >= 0.01:
        tendencia = 'creciente'
    elif slope <= -0.01:
        tendencia = 'decadente'
    else:
        tendencia = 'sin definir'

    clasificacion.append((genero, tendencia))

# Mostrar clasificación
df_resultado = pd.DataFrame(clasificacion, columns=['género', 'tendencia'])
print(df_resultado.sort_values('tendencia'))


                    género tendencia
0                   action   estable
16              simulation   estable
15                 shooter   estable
14  role-playing-games-rpg   estable
13                  racing   estable
12                  puzzle   estable
11              platformer   estable
10   massively-multiplayer   estable
17                  sports   estable
9                    indie   estable
7                   family   estable
6              educational   estable
5                   casual   estable
4                     card   estable
3              board-games   estable
2                   arcade   estable
1                adventure   estable
8                 fighting   estable
18                strategy   estable


In [ ]:
from scipy.stats import linregress
import pandas as pd

# Cargar y preparar los datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')
df = df.dropna(subset=['puntuacion_media', 'año', 'generos'])
df['año'] = df['año'].astype(int)
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

# Agrupar puntuaciones medias por año y género
grupo = df.groupby(['genero_principal', 'año'])['puntuacion_media'].mean().reset_index()

# Clasificar por pendiente
clasificacion = []
for genero in grupo['genero_principal'].unique():
    datos = grupo[grupo['genero_principal'] == genero]
    if len(datos['año'].unique()) < 5:
        clasificacion.append((genero, 'insuficiente'))
        continue
    slope, _, _, _, _ = linregress(datos['año'], datos['puntuacion_media'])
    if abs(slope) < 0.01:
        tendencia = 'estable'
    elif slope > 0.01:
        tendencia = 'creciente'
    else:
        tendencia = 'decadente'
    clasificacion.append((genero, tendencia))

# Crear DataFrame de clasificación
df_clasificacion = pd.DataFrame(clasificacion, columns=['genero_principal', 'tendencia'])

# Añadir tendencia al grupo original
grupo = grupo.merge(df_clasificacion, on='genero_principal')


In [ ]:
import plotly.express as px

# Elegimos una tendencia para graficar, por ejemplo: "creciente"
for tipo in ['creciente', 'estable', 'decadente']:
    datos = grupo[grupo['tendencia'] == tipo]

    if datos.empty:
        continue

    fig = px.line(
        datos,
        x='año',
        y='puntuacion_media',
        color='genero_principal',
        markers=True,
        title=f'⭐ Géneros con tendencia {tipo}',
        labels={
            'año': 'Año',
            'puntuacion_media': 'Puntuación media',
            'genero_principal': 'Género'
        }
    )
    fig.update_layout(template='plotly_dark')
    fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Cargar y preparar los datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')
df = df.dropna(subset=['año', 'generos'])
df['año'] = df['año'].astype(int)
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

# Agrupar por año y género para contar ocurrencias
conteo = df.groupby(['año', 'genero_principal']).size().reset_index(name='conteo')

# Para cada año, encontrar el género más frecuente (moda)
modas = conteo.sort_values('conteo', ascending=False).drop_duplicates('año')

# Ordenar cronológicamente
modas = modas.sort_values('año')

# Gráfico de barras para visualizar moda por año
fig = px.bar(
    modas,
    x='año',
    y='conteo',
    color='genero_principal',
    title='🎮 Género más popular por año (moda)',
    labels={
        'conteo': 'Número de juegos',
        'año': 'Año',
        'genero_principal': 'Género más frecuente'
    },
    hover_data=['genero_principal']
)

fig.update_layout(template='plotly_dark', xaxis=dict(dtick=1))
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Cargar y preparar datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')
df = df.dropna(subset=['generos', 'año'])
df['año'] = df['año'].astype(int)
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

# Contar número de juegos por género y año
conteo = df.groupby(['año', 'genero_principal']).size().reset_index(name='n_juegos')

# Crear gráfico animado
fig = px.bar(
    conteo,
    x='genero_principal',
    y='n_juegos',
    color='genero_principal',
    animation_frame='año',
    range_y=[0, conteo['n_juegos'].max() + 10],
    title='🕹 Evolución de géneros de videojuegos (1995–2024)',
    labels={
        'n_juegos': 'Número de juegos',
        'genero_principal': 'Género',
        'año': 'Año'
    }
)

fig.update_layout(template='plotly_dark', xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
import requests
import time
import csv

API_KEY = 'YOUR_API_KEY'  # ← REEMPLÁZALO
BASE_URL = 'https://api.rawg.io/api/games'

def obtener_juegos_desde_1995(paginas_por_año=3):
    juegos_por_año = []

    for año in range(1995, 2025):  # de 1995 a 2024 inclusive
        juegos_este_año = []
        for pagina in range(1, paginas_por_año + 1):
            params = {
                'key': API_KEY,
                'dates': f'{año}-01-01,{año}-12-31',
                'page': pagina,
                'page_size': 40,
                'ordering': '-rating'
            }
            response = requests.get(BASE_URL, params=params)
            if response.status_code != 200:
                print(f"Error {response.status_code} en el año {año}, página {pagina}")
                break

            data = response.json()
            for juego in data.get('results', []):
                rating = juego.get('rating')
                if rating and rating > 0 and juego.get('genres'):
                    juegos_este_año.append({
                        'nombre': juego.get('name'),
                        'fecha_lanzamiento': juego.get('released'),
                        'año': año,
                        'puntuacion_media': rating,
                        'generos': ', '.join([g['slug'] for g in juego['genres']])
                    })

            if not data.get('next'):
                break
            time.sleep(1)  # evitar bloqueo

        if not juegos_este_año:
            # Añadir entrada vacía si no hay datos para el año
            juegos_por_año.append({
                'nombre': 'Sin datos',
                'fecha_lanzamiento': None,
                'año': año,
                'puntuacion_media': None,
                'generos': 'sin_datos'
            })
        else:
            juegos_por_año.extend(juegos_este_año)

    return juegos_por_año

def guardar_csv(datos, archivo='videojuegos_1995_2024_completo.csv'):
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        campos = ['nombre', 'fecha_lanzamiento', 'año', 'puntuacion_media', 'generos']
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        writer.writerows(datos)

# Ejecutar
if __name__ == '__main__':
    juegos = obtener_juegos_desde_1995(paginas_por_año=3)
    guardar_csv(juegos)


Error 401 en el año 1995, página 1
Error 401 en el año 1996, página 1
Error 401 en el año 1997, página 1
Error 401 en el año 1998, página 1
Error 401 en el año 1999, página 1
Error 401 en el año 2000, página 1
Error 401 en el año 2001, página 1
Error 401 en el año 2002, página 1
Error 401 en el año 2003, página 1
Error 401 en el año 2004, página 1
Error 401 en el año 2005, página 1
Error 401 en el año 2006, página 1
Error 401 en el año 2007, página 1
Error 401 en el año 2008, página 1
Error 401 en el año 2009, página 1
Error 401 en el año 2010, página 1
Error 401 en el año 2011, página 1
Error 401 en el año 2012, página 1
Error 401 en el año 2013, página 1
Error 401 en el año 2014, página 1
Error 401 en el año 2015, página 1
Error 401 en el año 2016, página 1
Error 401 en el año 2017, página 1
Error 401 en el año 2018, página 1
Error 401 en el año 2019, página 1
Error 401 en el año 2020, página 1
Error 401 en el año 2021, página 1
Error 401 en el año 2022, página 1
Error 401 en el año 

In [ ]:
import requests
import time
import csv

API_KEY = '9e9ab06256174d82a01dc73f02418ca2'  # ← REEMPLÁZALO
BASE_URL = 'https://api.rawg.io/api/games'

def obtener_juegos_desde_1995(paginas_por_año=3):
    juegos_por_año = []

    for año in range(1995, 2025):  # de 1995 a 2024 inclusive
        juegos_este_año = []
        for pagina in range(1, paginas_por_año + 1):
            params = {
                'key': API_KEY,
                'dates': f'{año}-01-01,{año}-12-31',
                'page': pagina,
                'page_size': 40,
                'ordering': '-rating'
            }
            response = requests.get(BASE_URL, params=params)
            if response.status_code != 200:
                print(f"Error {response.status_code} en el año {año}, página {pagina}")
                break

            data = response.json()
            for juego in data.get('results', []):
                rating = juego.get('rating')
                if rating and rating > 0 and juego.get('genres'):
                    juegos_este_año.append({
                        'nombre': juego.get('name'),
                        'fecha_lanzamiento': juego.get('released'),
                        'año': año,
                        'puntuacion_media': rating,
                        'generos': ', '.join([g['slug'] for g in juego['genres']])
                    })

            if not data.get('next'):
                break
            time.sleep(1)  # evitar bloqueo

        if not juegos_este_año:
            # Añadir entrada vacía si no hay datos para el año
            juegos_por_año.append({
                'nombre': 'Sin datos',
                'fecha_lanzamiento': None,
                'año': año,
                'puntuacion_media': None,
                'generos': 'sin_datos'
            })
        else:
            juegos_por_año.extend(juegos_este_año)

    return juegos_por_año

def guardar_csv(datos, archivo='videojuegos_1995_2024_completo.csv'):
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        campos = ['nombre', 'fecha_lanzamiento', 'año', 'puntuacion_media', 'generos']
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        writer.writerows(datos)

# Ejecutar
if __name__ == '__main__':
    juegos = obtener_juegos_desde_1995(paginas_por_año=3)
    guardar_csv(juegos)


In [ ]:
import pandas as pd
import plotly.express as px

# Cargar y preparar los datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')
df = df.dropna(subset=['año', 'generos'])
df['año'] = df['año'].astype(int)
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

# Agrupar por año y género para contar ocurrencias
conteo = df.groupby(['año', 'genero_principal']).size().reset_index(name='conteo')

# Para cada año, encontrar el género más frecuente (moda)
modas = conteo.sort_values('conteo', ascending=False).drop_duplicates('año')

# Ordenar cronológicamente
modas = modas.sort_values('año')

# Gráfico de barras para visualizar moda por año
fig = px.bar(
    modas,
    x='año',
    y='conteo',
    color='genero_principal',
    title='🎮 Género más popular por año (moda)',
    labels={
        'conteo': 'Número de juegos',
        'año': 'Año',
        'genero_principal': 'Género más frecuente'
    },
    hover_data=['genero_principal']
)

fig.update_layout(template='plotly_dark', xaxis=dict(dtick=1))
fig.show()